In [19]:
import numpy as np
import csv
import sys
sys.path.append("..")
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from Code.VQLSSVM import VQLSSVM
from Code.Utils import prepareDataset

np.set_printoptions(precision=10, suppress=True)

In [20]:
qubits = 3
shots: int = 10000
gamma: float = 0.01 # regularization parameter
subsetSize: int = 2**qubits - 1 # number of training points
classToFilterOut: int = 2
iterations = 5
trainIterations = 200
# datasets = ["iris"]
datasets = ["iris","breastCancer"]

In [21]:
vqlssvmVectors: VQLSSVM = VQLSSVM(gamma, shots)

In [22]:
def getListsAverage(data: list):
    maximumLengthList = len(max(data, key=len))
    sumList =[]
    for i in range(maximumLengthList):
        iterationSum = 0
        for j in range(len(data)):
            if i < len(data[j]):
                iterationSum += data[j][i]
            else:
                iterationSum += data[j][-1]
        sumList.append(iterationSum)

    sumList = np.array(sumList)
    return sumList/len(data)

In [23]:
def collectTrainData(dataset: str, isQuantumSimulation: bool):
    xTrain, xTest, yTrain, yTest = prepareDataset(normalizeValues=True, dataset=dataset,subsetSize=subsetSize, classToFilterOut=classToFilterOut)
    vqlssvmVectors.train(xTrain, yTrain, quantumSimulation=isQuantumSimulation, verbose=False, iterations = trainIterations, method="COBYLA", threads=4, jobs=4)
    cost = vqlssvmVectors.getCostHistory()
    accuracy = vqlssvmVectors.accuracy(xTest, yTest)
    clf = SVC(kernel='linear')
    clf.fit(xTrain, yTrain)
    yPred = clf.predict(xTest)
    return cost, accuracy, accuracy_score(yTest, yPred)

costs = {}
accuracies = {}
accuraciesSVM = {}

for dataset in datasets:
    costs[dataset] = []
    accuracies[dataset] = []
    accuraciesSVM[dataset] = []

for i in range(iterations):
    print(i,"th iteration")
    for dataset in datasets:
        print("Dataset:",dataset)
        cost,accuracy,svmAccuracy = collectTrainData(dataset, True)
        costs[dataset].append(cost)
        accuracies[dataset].append(accuracy)
        accuraciesSVM[dataset].append(svmAccuracy)
        print("\n")
    print("\n")

0 th iteration
Dataset: iris
Transpiled circuits length: 666
Transpiled special circuits: 36
Iteration: 98 , cost: 0.00589171593790160916

Dataset: breastCancer
Transpiled circuits length: 666
Transpiled special circuits: 36
Iteration: 90 , cost: 0.0939207446848501244



1 th iteration
Dataset: iris
Transpiled circuits length: 666
Transpiled special circuits: 36
Iteration: 94 , cost: 0.01621468402760617445

Dataset: breastCancer
Transpiled circuits length: 666
Transpiled special circuits: 36
Iteration: 104 , cost: 0.05019346166892602



2 th iteration
Dataset: iris
Transpiled circuits length: 666
Transpiled special circuits: 36
Iteration: 92 , cost: 0.041029461244190684

Dataset: breastCancer
Transpiled circuits length: 666
Transpiled special circuits: 36
Iteration: 108 , cost: 0.0075926248905318076



3 th iteration
Dataset: iris
Transpiled circuits length: 666
Transpiled special circuits: 36
Iteration: 103 , cost: 0.0230233500377182796

Dataset: breastCancer
Transpiled circuits lengt

In [24]:
for dataset in datasets:
    with open('../SimulationResults/costDataset' + dataset+'.csv', 'w', newline='') as csvfile:
        avgCosts = {}
        length = costs[dataset].__len__()
        s = np.array([sum(a) for a in zip(*costs[dataset])])
        avgCosts[dataset] = getListsAverage(costs[dataset])
        
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(['Iteration'] + [dataset])
        combined = [ [i] + [x] for i, x in enumerate(avgCosts[dataset])]
        writer.writerows(combined)

with open('../SimulationResults/accuracyDataset.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    titleRow = []
    dataRow = []
    for dataset in datasets:
        titleRow.append(dataset)
        titleRow.append(dataset + "SVM")
        dataRow.append(np.mean(accuracies[dataset]))
        dataRow.append(np.mean(accuraciesSVM[dataset]))
    writer.writerow(titleRow)
    writer.writerow(dataRow)